# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [2]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [3]:
req = requests.get(url)
print(req.status_code)

200


## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(req.content, 'html.parser')
elements = soup.find_all('a',{'href':True})
links = list(set([element['href'] for element in elements if element['href'][0] != '#']))
# Removing hrefs begining with '#' because those are links to the same page but other location

In [6]:
# Estos links no tienen http y comienzan con / los pude haber filtrado, pero los dejé para ver qué pasaba :)
[link for link in links if link[:2] == '//']

['//www.wikimediafoundation.org/',
 '//www.worldcat.org/issn/0036-8075',
 '//creativecommons.org/licenses/by-sa/3.0/',
 '//foundation.wikimedia.org/wiki/Terms_of_Use',
 '//en.m.wikipedia.org/w/index.php?title=Data_science&mobileaction=toggle_view_mobile',
 '//pubmed.ncbi.nlm.nih.gov/19265007',
 '//en.wikipedia.org/wiki/Wikipedia:Text_of_Creative_Commons_Attribution-ShareAlike_3.0_Unported_License',
 '//www.worldcat.org/oclc/489990740',
 '//www.worldcat.org/issn/0017-8012',
 '//en.wikipedia.org/wiki/Wikipedia:Contact_us',
 '//foundation.wikimedia.org/wiki/Privacy_policy']

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
domain = 'http://wikipedia.org'

In [8]:
links_abs = [link for link in links if link[:4] == 'http' and '%' not in link]
links_rel = [f'{domain}{link}' for link in links if link[0] == '/' and '%' not in link]

In [9]:
links = list(set(links_abs + links_rel))

In [9]:
links

['http://wikipedia.org/wiki/Conditional_random_field',
 'http://wikipedia.org//www.worldcat.org/issn/0036-8075',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=12',
 'http://wikipedia.org/wiki/Statistical_learning_theory',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=9',
 'http://wikipedia.org/wiki/Decision_tree_learning',
 'https://magazine.amstat.org/blog/2015/10/01/asa-statement-on-the-role-of-statistics-in-data-science/',
 'http://wikipedia.org/wiki/Template_talk:Machine_learning_bar',
 'http://wikipedia.org/w/index.php?title=Special:CiteThisPage&page=Data_science&id=986108274&wpFormIdentifier=titleform',
 'http://wikipedia.org/wiki/Restricted_Boltzmann_machine',
 'http://wikipedia.org//www.wikimediafoundation.org/',
 'http://wikipedia.org/wiki/Wikipedia:About',
 'http://wikipedia.org/wiki/Data_curation',
 'http://wikipedia.org/wiki/Probably_approximately_correct_learning',
 'http://wikipedia.org/wiki/ISSN_(identifier)',
 'htt

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [10]:
import os

In [11]:
os.mkdir(f'{os.getcwd()}\wikipedia')

In [12]:
os.chdir(f'{os.getcwd()}\wikipedia')

In [13]:
os.getcwd()

'C:\\Users\\Angel\\Documents\\Ironhack\\datamex_102020\\module-1\\lab-parallelization\\your-code\\wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [14]:
from slugify import slugify

In [15]:
def index_page(link):

    try:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        file_name = f'{slugify(link)}.html'
        
        with open(file_name,'w', encoding='utf-8') as f:
            f.write(soup.prettify())

        print('.',end='')

    except:
        print(',',end='')
        pass

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [16]:
%%time
for link in links[:20]:
    index_page(link)

....................Wall time: 22.3 s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [17]:
import multiprocessing as mp

In [ ]:
# Este código corrió en colab pero no aquí ni en mi compu, pero aquí lo pego :) 
%%time
pool = mp.Pool(mp.cpu_count())
res = pool.map(index_page,links[:20])
pool.close()
res